In [2]:
import pandas as pd
import os
import glob
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

Can use simpler methods for fitting data for just one wavelength

In [3]:
data_dir = 'training_data/single_wave/'
all_data = glob.glob(os.path.join(data_dir, "*.csv"))
combined_data = pd.concat([pd.read_csv(file) for file in all_data], ignore_index=True)
combined_data.drop(["Unnamed: 0"], axis = 1)

,Wavelength,V1,V2,Gain,Out
0,490,0.600000,0.600000,4,-0.245645
1,490,0.684685,0.600000,4,-0.245820
2,490,0.769369,0.600000,4,-0.246121
3,490,0.854054,0.600000,4,-0.246086
4,490,0.938739,0.600000,4,-0.246285
...,...,...,...,...,...
1830,490,1.037209,9.562791,4,-0.110316
1831,490,0.927907,9.672093,4,-0.098668
1832,490,0.818605,9.781395,4,-0.098445
1833,490,0.709302,9.890698,4,-0.090430


In [ ]:
x = combined_data['V1']
y = combined_data['V2']
z = combined_data['']